grasp

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/12/25 16:19:26.927 8542] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
gs.init(backend=gs.gpu, seed=42)
adapter = GenesisAdapter()

[Genesis] [16:19:32] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [16:19:32] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [16:19:32] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [16:19:33] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [16:19:33] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: 42, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [16:19:34] [INFO] Scene <6a9746c> created.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)
# robot_path = "../assets/urdf/franka_description/robots/frankaEmikaPandaVirtual.urdf"
# franka_virtual = scene.add_entity(
#     gs.morphs.URDF(file=robot_path, fixed=True),
# )

franka_MJCF = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml",
                   pos = (0.0, 2.0, 0.0)),
    
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda.xml",)
                   
)
cube_size = (0.1, 0.07, 0.03)
cube_pos = [(0.7, 0, 0.015), (0.7, 2.0, 0.015), (0.7, -2.0, 0.015)]
cube, cube2, cube3 = [scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = pos,
    ),
    surface=gs.surfaces.Default(
        color=(0.8, 0.1, 0.1),

    ),
    material=gs.materials.Rigid(friction=0.01)
    
) for pos in cube_pos]

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )


[Genesis] [16:19:36] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <86b31a5>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [16:19:36] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <a82990c>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [16:19:37] [WARNING] (MJCF) Tendon not supported
[Genesis] [16:19:37] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [16:19:37] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [16:19:37] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [16:19:37] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [16:19:37] [INFO] Appl

In [4]:
n_envs = 0
scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

[Genesis] [16:19:50] [INFO] Building scene <6a9746c>...
[Genesis] [16:19:51] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [16:19:51] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/12/25 16:19:53.220 8542] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [16:19:59] [INFO] Compiling simulation kernels...
[Genesis] [16:22:58] [INFO] Building visualizer...
[Genesis] [16:23:01] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner

planner = ScrewMotionPlanner()

In [6]:
bbox = adapter.get_bbox(cube)
planner.find_ground_edges(bbox)
qs, s_axes = planner.screw_from_bbox(bbox=bbox)

In [7]:
pt.random_screw_axis

<function pytransform3d.transformations._random.random_screw_axis(rng=Generator(PCG64) at 0x7EF47BBBCC80)>

In [8]:
s_axes

array([[0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [9]:
scene.reset()

[Genesis] [15:46:47] [INFO] Resetting Scene <ad61470>.


In [5]:
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

franka.ready_qpos = qr
franka.set_qpos(franka.ready_qpos)
franka.control_dofs_position(franka.ready_qpos)
franka_MJCF.set_qpos(franka.ready_qpos)
franka_MJCF.control_dofs_position(franka.ready_qpos)

end_effector = franka.get_link('hand')

scene.step()

[Genesis] [16:24:18] [INFO] Running at 0.26 FPS.


In [11]:
for _ in range(100):
    scene.step()

[Genesis] [15:46:50] [INFO] Running at 0.92 FPS.
[Genesis] [15:46:50] [INFO] Running at 0.96 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.01 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.06 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.12 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.18 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.24 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.30 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.37 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.44 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.51 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.59 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.67 FPS.
[Genesis] [15:46:50] [INFO] Running at 1.75 FPS.
[Genesis] [15:46:51] [INFO] Running at 1.84 FPS.
[Genesis] [15:46:51] [INFO] Running at 1.93 FPS.
[Genesis] [15:46:51] [INFO] Running at 2.03 FPS.
[Genesis] [15:46:51] [INFO] Running at 2.13 FPS.
[Genesis] [15:46:51] [INFO] Running at 2.24 FPS.
[Genesis] [15:46:51] [INFO] Running at 2.35 FPS.
[Genesis] [15:46:51]

In [6]:
target_entity.set_pos(end_effector.get_pos())
target_entity.set_quat(end_effector.get_quat())
scene.step()

[Genesis] [16:24:27] [INFO] Running at 0.25 FPS.


In [ ]:
# target_quat = np.array([0, 1, 0, 0])
# center = np.array([0.5, 0, 0.5])
# r = 0.1
# damping = 1e-6
# diag = damping * np.eye(6)
# target_pos = center + np.array([1, 0, 0]) * r
# init_q = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = target_pos,
#     quat = target_quat,
# )
# franka.set_qpos(init_q)
# franka.control_dofs_position(init_q)
# for i in range(100):
#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))
#     scene.step()

# for i in range(0, 500):
#     target_pos = center + np.array([np.cos(i / 360 * np.pi), np.sin(i / 360 * np.pi), 0]) * r

#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))

#     # Position error.
#     error_pos = target_pos - end_effector.get_pos().cpu().numpy()

#     # Orientation error.
#     ee_quat = end_effector.get_quat().cpu().numpy()
#     error_quat = gs.transform_quat_by_quat(gs.inv_quat(ee_quat), target_quat)
#     error_rotvec = gs.quat_to_rotvec(error_quat)

#     error = np.concatenate([error_pos, error_rotvec])

#     # jacobian
#     jac = franka.get_jacobian(link=end_effector).cpu().numpy()
#     dq = jac.T @ np.linalg.solve(jac @ jac.T + diag, error)
#     q = franka.get_qpos().cpu().numpy() + dq

#     # control
#     franka.control_dofs_position(q)
#     scene.step()


In [7]:

# get the end-effector link
end_effector = franka.get_link('hand')
end_effector_MJCF = franka_MJCF.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[0]/2 + 0.02, 0, 0.11 + 0.1])
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[-2:] = 0.04
qpos_MJCF[-2:] = 0.04
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_MJCF,
    num_waypoints = 200, # 2s duration
)

[W 05/12/25 16:24:32.645 8542] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.
[W 05/12/25 16:24:42.783 8542] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [16:24:53] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [16:24:53] [INFO] Path solution found successfully.
[Genesis] [16:24:53] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [16:24:53] [INFO] Path solution found successfully.


In [8]:
# execute the planned path
for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


[Genesis] [16:25:57] [INFO] Running at 0.12 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.13 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.13 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.14 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.15 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.15 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.16 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.17 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.18 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.19 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.20 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.21 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.22 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.23 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.24 FPS.
[Genesis] [16:25:57] [INFO] Running at 0.26 FPS.
[Genesis] [16:25:58] [INFO] Running at 0.27 FPS.
[Genesis] [16:25:58] [INFO] Running at 0.28 FPS.
[Genesis] [16:25:58] [INFO] Running at 0.30 FPS.
[Genesis] [16:25:58] [INFO] Running at 0.31 FPS.
[Genesis] [16:25:58]

In [9]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = end_effector,
    pos  = side_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_reach_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = side_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
path = franka.plan_path(
    qpos_goal     = qpos_reach,
    num_waypoints = 100, # 1s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_reach_MJCF,
    num_waypoints = 100, # 1s duration
)

for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [16:27:28] [INFO] Path solution found successfully.
[Genesis] [16:27:29] [INFO] Path solution found successfully.
[Genesis] [16:27:29] [INFO] Running at 0.24 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.26 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.27 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.28 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.30 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.31 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.33 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.35 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.37 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.39 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.41 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.43 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.45 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.47 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.50 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.52 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.55 FPS.
[Genesis] [16:27:29] [INFO] Running at 0.58

In [10]:



# grasp
franka.control_dofs_position(qpos_reach[:-2], motors_dof)
franka.control_dofs_force(np.array([-5, -5]), fingers_dof)
franka_MJCF.control_dofs_position(qpos_reach_MJCF[:-2], motors_dof)
franka_MJCF.control_dofs_force(np.array([-5, -5]), fingers_dof)

for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


[Genesis] [16:27:52] [INFO] Running at 1.17 FPS.
[Genesis] [16:27:52] [INFO] Running at 1.23 FPS.
[Genesis] [16:27:52] [INFO] Running at 1.28 FPS.
[Genesis] [16:27:52] [INFO] Running at 1.33 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.39 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.45 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.51 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.58 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.65 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.72 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.79 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.87 FPS.
[Genesis] [16:27:53] [INFO] Running at 1.95 FPS.
[Genesis] [16:27:54] [INFO] Running at 2.03 FPS.
[Genesis] [16:27:54] [INFO] Running at 2.11 FPS.
[Genesis] [16:27:54] [INFO] Running at 2.20 FPS.
[Genesis] [16:27:54] [INFO] Running at 2.29 FPS.
[Genesis] [16:27:54] [INFO] Running at 2.38 FPS.
[Genesis] [16:27:54] [INFO] Running at 2.47 FPS.
[Genesis] [16:27:54] [INFO] Running at 2.57 FPS.
[Genesis] [16:27:54]

In [11]:

# lift
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [16:28:09] [INFO] Running at 2.14 FPS.
[Genesis] [16:28:09] [INFO] Running at 2.23 FPS.
[Genesis] [16:28:10] [INFO] Running at 2.33 FPS.
[Genesis] [16:28:10] [INFO] Running at 2.42 FPS.
[Genesis] [16:28:10] [INFO] Running at 2.52 FPS.
[Genesis] [16:28:10] [INFO] Running at 2.63 FPS.
[Genesis] [16:28:10] [INFO] Running at 2.75 FPS.
[Genesis] [16:28:10] [INFO] Running at 2.87 FPS.
[Genesis] [16:28:10] [INFO] Running at 2.99 FPS.
[Genesis] [16:28:10] [INFO] Running at 3.11 FPS.
[Genesis] [16:28:10] [INFO] Running at 3.22 FPS.
[Genesis] [16:28:10] [INFO] Running at 3.33 FPS.
[Genesis] [16:28:10] [INFO] Running at 3.45 FPS.
[Genesis] [16:28:10] [INFO] Running at 3.57 FPS.
[Genesis] [16:28:11] [INFO] Running at 3.68 FPS.
[Genesis] [16:28:11] [INFO] Running at 3.80 FPS.
[Genesis] [16:28:11] [INFO] Running at 3.92 FPS.
[Genesis] [16:28:11] [INFO] Running at 4.05 FPS.
[Genesis] [16:28:11] [INFO] Running at 4.17 FPS.
[Genesis] [16:28:11] [INFO] Running at 4.28 FPS.
[Genesis] [16:28:11]

In [13]:
adapter.step_simulation(5)

[Genesis] [16:06:28] [INFO] Running at 0.73 FPS.
[Genesis] [16:06:29] [INFO] Running at 0.77 FPS.
[Genesis] [16:06:29] [INFO] Running at 0.80 FPS.
[Genesis] [16:06:29] [INFO] Running at 0.84 FPS.
[Genesis] [16:06:29] [INFO] Running at 0.88 FPS.
[Genesis] [16:06:29] [INFO] Running at 0.93 FPS.
[Genesis] [16:06:29] [INFO] Running at 0.97 FPS.
[Genesis] [16:06:29] [INFO] Running at 1.02 FPS.
[Genesis] [16:06:29] [INFO] Running at 1.06 FPS.
[Genesis] [16:06:29] [INFO] Running at 1.11 FPS.
[Genesis] [16:06:29] [INFO] Running at 1.17 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.22 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.28 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.34 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.40 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.46 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.53 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.60 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.67 FPS.
[Genesis] [16:06:30] [INFO] Running at 1.74 FPS.
[Genesis] [16:06:30]

In [40]:
ff = 0
franka.control_dofs_force(np.array([ff, ff]), fingers_dof)
franka_MJCF.control_dofs_force(np.array([ff, ff]), fingers_dof)
for _ in range(500):
    scene.step()

[Genesis] [01:50:38] [INFO] Running at 0.10 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.10 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.11 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.11 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.12 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.13 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.13 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.14 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.15 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.15 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.16 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.17 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.18 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.19 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.20 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.21 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.22 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.23 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.25 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.26 FPS.
[Genesis] [01:50:39]